In [2]:
from create_overview import ResultAggregator
from copy import deepcopy
import re
from ast import literal_eval
from statistics import mean
import numpy as np
from collections import defaultdict
import pandas as pd

In [3]:
wandb_api_key = "3b7e4cee8259b9c6522aa3adef3833e6c109f000"

In [4]:
#ra = ResultAggregator(wandb_api_key=wandb_api_key, verbose_logging=False)
ra = ResultAggregator(path_to_csv_export="current_wandb_results_unprocessed.csv", verbose_logging=False)
#ra.get_info()

/Users/test/Documents/GitHub/LEXTREME/utils/create_overview.py:78: DtypeWarning: Columns (23,32,60,81,95,177,220,221) have mixed types. Specify dtype option on import or set low_memory=False.
  results = pd.read_csv(Path(path_to_csv_export))


In [11]:
','.join(ra.meta_infos['task_language_mapping']['mapa_coarse'])

'it,sk,mt,es,en,el,de,pt,ro,lt,ga,fi,bg,nl,fr,et,hu,lv,da,cs,sv'

In [9]:
for lm, language in ra.meta_infos['model_language_lookup_table'].items():
    if language =="pt":
        print(lm)

neuralmind/bert-base-portuguese-cased


In [8]:
for model, language in ra.meta_infos['model_language_lookup_table'].items():
    #print(language)
    if language in ra.meta_infos['task_language_mapping']['lener_br']:
        print(model)

neuralmind/bert-base-portuguese-cased


In [18]:
task_constraint = ["brazilian_court_decisions_judgment",  "brazilian_court_decisions_unanimity", "german_argument_mining", "covid19_emergency_event", "online_terms_of_service_clause_topics"]
print(task_constraint)

['brazilian_court_decisions_judgment', 'brazilian_court_decisions_unanimity', 'german_argument_mining', 'covid19_emergency_event', 'online_terms_of_service_clause_topics']


In [32]:
task = "covid19_emergency_event"
relevant_languages = ra.meta_infos['task_language_mapping'][task]
relevant_columns = [l+'_predict/_micro-f1' for l in relevant_languages]

In [33]:
ra.results[ra.results.finetuning_task=="covid19_emergency_event"][['_name_or_path','predict/_macro-f1']+relevant_columns]

,_name_or_path,predict/_macro-f1,hu_predict/_micro-f1,it_predict/_micro-f1,pl_predict/_micro-f1,nl_predict/_micro-f1,en_predict/_micro-f1,fr_predict/_micro-f1,nb_predict/_micro-f1
220,GroNLP/bert-base-dutch-cased,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,GroNLP/bert-base-dutch-cased,0.041667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,GroNLP/bert-base-dutch-cased,0.041667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
772,xlm-roberta-large,0.525693,0.666667,0.800000,0.941176,0.600000,0.727273,0.728205,0.594595
774,xlm-roberta-large,0.442743,0.500000,0.828125,0.750000,0.625000,0.711864,0.729167,0.470588
775,xlm-roberta-base,0.356741,0.000000,0.699187,0.875000,0.555556,0.654867,0.699454,0.514286
777,xlm-roberta-base,0.359816,0.666667,0.736000,0.875000,0.500000,0.601770,0.724490,0.411765
779,microsoft/mdeberta-v3-base,0.404276,0.000000,0.731707,0.705882,0.588235,0.660550,0.743455,0.400000
781,xlm-roberta-large,0.581079,0.000000,0.791045,0.666667,0.500000,0.660870,0.738462,0.700000
782,microsoft/mdeberta-v3-base,0.454199,0.000000,0.787402,0.888889,0.526316,0.678899,0.709360,0.457143


In [19]:
ra.reset_list_of_available_languages()
print('Results without all available languages:')
print('------------------------')
ra.get_language_aggregated_score(task_constraint=task_constraint)
#ra.get_dataset_aggregated_score()
ra.get_dataset_aggregated_score(average_over_language=False, task_constraint=task_constraint)

lm = ["distilbert-base-multilingual-cased", "microsoft/Multilingual-MiniLM-L12-H384", "xlm-roberta-base", "microsoft/mdeberta-v3-base", "xlm-roberta-large"]

for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')


Results without all available languages:
------------------------
distilbert-base-multilingual-cased
	 language aggregated:  0.30521488431702043
	 dataset aggregated:  0.48887447234257053
-----------------------------------
microsoft/Multilingual-MiniLM-L12-H384
	 language aggregated:  0.15185585211365582
	 dataset aggregated:  0.37185766557129846
-----------------------------------
xlm-roberta-base
	 language aggregated:  0.3670587596860509
	 dataset aggregated:  0.5703489759508551
-----------------------------------
microsoft/mdeberta-v3-base
	 language aggregated:  0.35614228392372016
	 dataset aggregated:  0.5579436038193932
-----------------------------------
xlm-roberta-large
	 language aggregated:  0.40559579539596297
	 dataset aggregated:  0.6503324188980668
-----------------------------------

##########################



In [23]:
ra.reset_list_of_available_languages()
print('Results without all available languages:')
print('------------------------')
ra.get_language_aggregated_score(task_constraint=task_constraint)
ra.get_dataset_aggregated_score(task_constraint=task_constraint)
#ra.get_dataset_aggregated_score(average_over_language=False)

lm = ["distilbert-base-multilingual-cased", "microsoft/Multilingual-MiniLM-L12-H384", "xlm-roberta-base", "microsoft/mdeberta-v3-base", "xlm-roberta-large"]

for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')


Results without all available languages:
------------------------
distilbert-base-multilingual-cased
	 language aggregated:  0.30521488431702043
	 dataset aggregated:  0.4440381749257066
-----------------------------------
microsoft/Multilingual-MiniLM-L12-H384
	 language aggregated:  0.15185585211365582
	 dataset aggregated:  0.3639526551474359
-----------------------------------
xlm-roberta-base
	 language aggregated:  0.3670587596860509
	 dataset aggregated:  0.5260967546428402
-----------------------------------
microsoft/mdeberta-v3-base
	 language aggregated:  0.35614228392372016
	 dataset aggregated:  0.5166805130910594
-----------------------------------
xlm-roberta-large
	 language aggregated:  0.40559579539596297
	 dataset aggregated:  0.571635716778012
-----------------------------------

##########################



In [22]:
ra.reset_list_of_available_languages()
print('Results without all available languages:')
print('------------------------')
ra.get_language_aggregated_score(task_constraint=task_constraint)
ra.get_dataset_aggregated_score(task_constraint=task_constraint)

lm = ["distilbert-base-multilingual-cased", "microsoft/Multilingual-MiniLM-L12-H384", "xlm-roberta-base", "microsoft/mdeberta-v3-base", "xlm-roberta-large"]

for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')

print('Results without Hungarian:')
print('------------------------')
ra.remove_languages(['hu'])
ra.get_language_aggregated_score(task_constraint=task_constraint)
ra.get_dataset_aggregated_score(task_constraint=task_constraint)


for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')


ra.reset_list_of_available_languages()

print('Results without Dutch:')
print('------------------------')
ra.remove_languages(['nl'])
ra.get_language_aggregated_score(task_constraint=task_constraint)
ra.get_dataset_aggregated_score(task_constraint=task_constraint)


for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')

ra.reset_list_of_available_languages()

print('Results without Dutch and Hungarian:')
print('------------------------')
ra.remove_languages(['nl','hu'])
ra.get_language_aggregated_score(task_constraint=task_constraint)
ra.get_dataset_aggregated_score(task_constraint=task_constraint)


for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')


Results without all available languages:
------------------------
distilbert-base-multilingual-cased
	 language aggregated:  0.30521488431702043
	 dataset aggregated:  0.4440381749257066
-----------------------------------
microsoft/Multilingual-MiniLM-L12-H384
	 language aggregated:  0.15185585211365582
	 dataset aggregated:  0.3639526551474359
-----------------------------------
xlm-roberta-base
	 language aggregated:  0.3670587596860509
	 dataset aggregated:  0.5260967546428402
-----------------------------------
microsoft/mdeberta-v3-base
	 language aggregated:  0.35614228392372016
	 dataset aggregated:  0.5166805130910594
-----------------------------------
xlm-roberta-large
	 language aggregated:  0.40559579539596297
	 dataset aggregated:  0.571635716778012
-----------------------------------

##########################

Results without Hungarian:
------------------------
distilbert-base-multilingual-cased
	 language aggregated:  0.343366744856648
	 dataset aggregated:  0.4532326

In [29]:
from datasets import load_dataset

dataset = load_dataset("joelito/lextreme", task)
train_set = pd.DataFrame(dataset['train'])
print(train_set.language.unique())

  0%|          | 0/3 [00:00<?, ?it/s]

['fr' 'hu' 'it' 'nl' 'nb' 'pl' 'en']


In [36]:
train_set[train_set.language=="en"].shape

(648, 3)

In [ ]:
train_set

In [18]:
language = "nb"
train_set[train_set.language==language]

,input,label,language
2368,Ikke reis hvis du er syk: Hvis du er syk eller...,[3],nb
2369,lt det vi gjør nå er for at folk skal få god b...,[],nb
2370,Dette innebærer også at treninger og annen org...,"[1, 2]",nb
2371,Lettelse i innreiserestriksjoner for arbeidsin...,[4],nb
2372,Reglene om bortvisning av utenlandske statsbor...,[6],nb
...,...,...,...
2584,Takket være vår felles innsats siden mars har ...,[],nb
2585,Flyene har gått med nordmenn og andre nordiske...,[],nb
2586,Denne bestemmelsen forutsetter at regjeringen ...,[],nb
2587,Bruk sykkel og andre transportmidler: Kan du s...,[],nb


In [20]:
[l for l in train_set[train_set.language==language].label.tolist() if l!=[]]

[[3],
 [1, 2],
 [4],
 [6],
 [1, 4],
 [2],
 [3],
 [1, 2],
 [2],
 [3],
 [2],
 [3],
 [3],
 [6],
 [6],
 [2],
 [2],
 [4],
 [4],
 [3],
 [0],
 [3],
 [6],
 [2],
 [4],
 [3],
 [4],
 [6],
 [2],
 [2],
 [3],
 [0],
 [4],
 [4],
 [3],
 [2],
 [3],
 [3],
 [7],
 [3],
 [6],
 [3],
 [3],
 [2],
 [2],
 [3],
 [6],
 [6],
 [6],
 [3],
 [3],
 [3],
 [4],
 [4],
 [3],
 [3],
 [6],
 [3, 4],
 [2],
 [4],
 [3],
 [3],
 [4],
 [4],
 [3],
 [6],
 [2],
 [2],
 [4],
 [2],
 [2],
 [0],
 [2],
 [6],
 [6],
 [1, 2],
 [6],
 [2],
 [4],
 [3],
 [2, 3],
 [4],
 [6],
 [1, 2],
 [4],
 [0],
 [3],
 [2],
 [3, 7],
 [0],
 [4],
 [2],
 [2],
 [0],
 [3],
 [3],
 [6],
 [4],
 [3, 7],
 [3],
 [2],
 [3],
 [0],
 [6],
 [3],
 [2],
 [2],
 [6],
 [3],
 [2, 6],
 [3],
 [4],
 [2, 3],
 [4],
 [2],
 [4],
 [4],
 [3],
 [0],
 [3],
 [4],
 [3],
 [6],
 [7],
 [3],
 [3]]

In [12]:
import json as js

with open('../meta_infos.json', 'r') as f:
    data = js.load(f)
data

{'task_code_mapping': {'brazilian_court_decisions_judgment': 'SLTC',
  'brazilian_court_decisions_unanimity': 'SLTC',
  'german_argument_mining': 'SLTC',
  'greek_legal_code_chapter': 'SLTC',
  'greek_legal_code_subject': 'SLTC',
  'greek_legal_code_volume': 'SLTC',
  'swiss_judgment_prediction': 'SLTC',
  'online_terms_of_service_unfairness_levels': 'SLTC',
  'online_terms_of_service_clause_topics': 'MLTC',
  'covid19_emergency_event': 'MLTC',
  'multi_eurlex_level_1': 'MLTC',
  'multi_eurlex_level_2': 'MLTC',
  'multi_eurlex_level_3': 'MLTC',
  'greek_legal_ner': 'NER',
  'legalnero': 'NER',
  'lener_br': 'NER',
  'mapa_coarse': 'NER',
  'mapa_fine': 'NER'},
 'task_language_mapping': {'brazilian_court_decisions_judgment': ['pt'],
  'brazilian_court_decisions_unanimity': ['pt'],
  'german_argument_mining': ['de'],
  'greek_legal_code_chapter': ['el'],
  'greek_legal_code_subject': ['el'],
  'greek_legal_code_volume': ['el'],
  'swiss_judgment_prediction': ['de', 'fr', 'it'],
  'online

In [13]:
# Creating a dictionary to look up the language for each language model 
model_language_lookup_table = dict()
for k, v in data["language_models"].items():
    if type(v) == dict:
        for language, info in v.items():
            if language == "multilingual":
                language = "all"
            for size, models in info.items():
                for m in models:
                    model_language_lookup_table[m] = language


In [16]:
model_language_lookup_table

{'iarfmoose/roberta-base-bulgarian': 'bg',
 'Maltehb/danish-bert-botxo': 'da',
 'dbmdz/bert-base-german-cased': 'de',
 'deepset/gbert-base': 'de',
 'nlpaueb/bert-base-greek-uncased-v1': 'el',
 'roberta-base': 'en',
 'bertin-project/bertin-roberta-base-spanish': 'es',
 'PlanTL-GOB-ES/roberta-base-bne': 'es',
 'TurkuNLP/bert-base-finnish-cased-v1': 'fi',
 'camembert-base': 'fr',
 'dbmdz/bert-base-french-europeana-cased': 'fr',
 'DCU-NLP/bert-base-irish-cased-v1': 'ga',
 'Musixmatch/umberto-commoncrawl-cased-v1': 'it',
 'dbmdz/bert-base-italian-cased': 'it',
 'GroNLP/bert-base-dutch-cased': 'nl',
 'dkleczek/bert-base-polish-uncased-v1': 'pl',
 'neuralmind/bert-base-portuguese-cased': 'pt',
 'dumitrescustefan/bert-base-romanian-uncased-v1': 'ro',
 'gerulata/slovakbert': 'sk',
 'KB/bert-base-swedish-cased': 'sv',
 'distilbert-base-multilingual-cased': 'all',
 'microsoft/Multilingual-MiniLM-L12-H384': 'all',
 'xlm-roberta-base': 'all',
 'microsoft/mdeberta-v3-base': 'all',
 'xlm-roberta-larg